In [1]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.metrics import log_loss

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
train_imputed = pd.read_csv('train_imputed.csv')
test_imputed = pd.read_csv('test_imputed.csv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


184.8537561893463


In [4]:
train.shape

(250000, 237)

In [5]:
train_imputed.isnull().any().any()

False

In [6]:
test_imputed.shape

(1888195, 42)

In [7]:
train_imputed.shape

(226140, 42)

In [8]:
train_imputed = train_imputed.drop('Unnamed: 0',axis=1)
test_imputed = test_imputed.drop('Unnamed: 0',axis=1)

In [9]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0013065338134765625


In [10]:
train['renewed_yorn'].isnull().sum()

23860

In [11]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.5344786643981934


In [12]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop(['renewed_yorn','zip_code'],axis=1)
test_y=test['renewed_yorn']
test = test.drop(['renewed_yorn','zip_code'],axis=1)
t2=time.time()
print(t2-t1)

4.189247131347656


In [13]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
clean_columns=test.columns[percent==0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

46.94104599952698


In [14]:
len(unclean_columns)

36

In [15]:
len(clean_columns)

133

In [16]:
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
target_columns=test.columns[percent>0]
train_missing=train[target_columns]
test_missing=test[target_columns]
train = train.drop(target_columns,axis=1)
test = test.drop(target_columns,axis=1)

In [17]:
len(target_columns)

66

In [18]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in target_columns:
    if(train_missing[col].dtype=='O' and len(train_missing[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.9067142009735107


In [19]:
# 56 messy columns are removed
len(messy_columns)

25

In [20]:
len(train_missing.columns)

66

In [21]:
t1=time.time()
train_missing=train_missing.drop(messy_columns,axis=1)
test_missing=test_missing.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

1.0682344436645508


In [22]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.9743523597717285


In [23]:
len(messy_columns)

61

In [24]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

2.0690605640411377


In [25]:
train.shape

(226140, 72)

In [26]:
train_missing.shape

(226140, 41)

In [27]:
train_imputed.shape

(226140, 41)

In [28]:
train_imputed.isnull().any().any()

False

In [29]:
train.isnull().any().any()

False

In [30]:
train.columns

Index(['innovation_challenge_key', 'minor_line_yorn', 'instance_status',
       'product_transaction_type', 'installed_product_type',
       'installed_product_component_yorn', 'component_type',
       'scdc_product_yorn', 'service_classification',
       'contract_line_multi_year_discount_yorn',
       'contract_line_route_to_market_code', 'contract_line_sales_motion_code',
       'default_multi_year_discount_yorn', 'low_dollar_contract_yorn',
       'contract_line_source_business_process_name',
       'contract_line_repair_service_order_level',
       'contract_line_reaction_time_code', 'product_route_to_market',
       'service_route_to_market_code', 'auto_quote_yorn', 'hardware_yorn',
       'software_yorn', 'acw_yorn', 'enterprise_license_agreement_yorn',
       'software_opportunity_type', 'sales_hierarchy_level',
       'united_states_yorn', 'level_1_name',
       'service_sales_node_base_sales_hierarchy_level',
       'service_sales_node_base_scms_name',
       'service_sales_n

In [31]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop(['innovation_challenge_key'],axis=1)
test_ids=test['innovation_challenge_key']
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.3237998485565186


In [32]:
train.isnull().any().any()

False

In [33]:
train=train.reset_index()

In [34]:
train_imputed = train_imputed.drop('index',axis=1)

ValueError: labels ['index'] not contained in axis

In [35]:
for col in train_imputed.columns:
    train[col]=train_imputed[col]
    test[col]=test_imputed[col]

In [36]:
train[col].shape

(226140,)

In [37]:
train.shape

(226140, 113)

In [38]:
test.shape

(1888195, 112)

In [39]:
train.isnull().any().any()

False

In [47]:
train = train.drop('index',axis=1)

In [48]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.042971134185791016


In [49]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.009778022766113281


In [50]:
train.isnull().any().sum()

0

In [51]:
test.isnull().any().any()

False

In [52]:
train_imputed.columns

Index(['service_contract_process_type',
       'product_bookings_measure_transaction_type', 'product_adjustment_type',
       'product_channel_bookings_yorn', 'service_channel_bookings_yorn',
       'service_bookings_measure_transaction_type', 'scms_name',
       'country_type', 'service_product_base_product_family_owner',
       'service_product_base_service_brand_code', 'status_code',
       'partner_country_registered_type', 'type_description', 'partner_tier',
       'certification_description', 'qualification_type_description',
       'disadvantaged_business_yorn', 'minority_owned_business_yorn',
       'woman_owned_business_yorn', 'business_owner_ethnicity_code',
       'hubzone_small_business_yorn', 'disabled_veteran_owned_business_yorn',
       'service_partner_base_status_code',
       'service_partner_base_partner_country_registered_type',
       'service_partner_base_type_description',
       'service_partner_base_partner_tier',
       'service_partner_base_certification_desc

In [53]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

2.2128138542175293


In [54]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05
mean,1.005455e-17,2.815275e-17,-2.915821e-17,4.775913e-18,3.016366e-18,5.027277e-18,-9.426144e-20
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.252316e-01,-2.321716e-01,-2.737838e+00,-1.742868e-01,-7.790595e-02,-1.128741e+00,-5.226844e-02
25%,-1.252316e-01,-2.321716e-01,-6.842019e-01,-1.053205e-01,-7.719748e-02,-2.906674e-01,-2.610574e-02
50%,-1.252316e-01,-2.321716e-01,-7.744580e-02,-4.824491e-02,-7.436361e-02,-1.738249e-02,-1.613900e-02
75%,-1.252316e-01,-1.584198e-01,2.959425e-01,6.452532e-03,-6.515353e-02,1.283695e-01,-1.188889e-03
max,7.774224e+01,6.648806e+01,4.118197e+01,9.915151e+01,1.359952e+02,4.662324e+01,1.670925e+02


In [55]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,226140,226140,226140,226140
unique,3,3,6,6
top,10002,-1,6,6
freq,202522,225296,199960,224170


In [56]:
for col in train.columns:
    print(col+' '+str(len(train[col].unique())))

minor_line_yorn 2
instance_status 1
product_transaction_type 3
installed_product_type 2
installed_product_component_yorn 2
component_type 4
scdc_product_yorn 1
service_classification 2
contract_line_multi_year_discount_yorn 2
contract_line_route_to_market_code 3
contract_line_sales_motion_code 1
default_multi_year_discount_yorn 2
low_dollar_contract_yorn 2
contract_line_source_business_process_name 2
contract_line_repair_service_order_level 2
contract_line_reaction_time_code 3
product_route_to_market 5
service_route_to_market_code 5
auto_quote_yorn 2
hardware_yorn 2
software_yorn 2
acw_yorn 2
enterprise_license_agreement_yorn 2
software_opportunity_type 3
sales_hierarchy_level 6
united_states_yorn 2
level_1_name 5
service_sales_node_base_sales_hierarchy_level 6
service_sales_node_base_scms_name 6
service_sales_node_base_united_states_yorn 2
service_sales_node_base_country_type 3
service_sales_node_base_level_1_name 6
item_type 5
manufacturing_plant 2
software_stack 4
monetization_type 

In [57]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

21.168502807617188


In [58]:
train.shape

(226140, 282)

In [59]:
test.shape

(1888195, 282)

In [60]:
train.isnull().any().any()

False

In [61]:
test.isnull().any().any()

False

In [62]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.41207385063171387


In [63]:
X_train.shape

(151513, 282)

In [64]:
import gc
del dataset
gc.collect()

2028

In [108]:
#use knn to group similar rows
from sklearn.cluster import KMeans
t1=time.time()
kmeans = KMeans(n_clusters=5, random_state=0).fit(X_train)
X_train_labels = kmeans.labels_
t2=time.time()
print(t2-t1)

38.19946885108948


In [113]:
X_train_label0 = X_train[X_train_labels==0]

In [114]:
X_train_label1 = X_train[X_train_labels==1]
X_train_label2 = X_train[X_train_labels==2]
X_train_label3 = X_train[X_train_labels==3]
X_train_label4 = X_train[X_train_labels==4]

In [119]:
y_train_label0 = y_train[X_train_labels==0]
y_train_label1 = y_train[X_train_labels==1]
y_train_label2 = y_train[X_train_labels==2]
y_train_label3 = y_train[X_train_labels==3]
y_train_label4 = y_train[X_train_labels==4]

In [115]:
X_valid_labels = kmeans.predict(X_valid)

In [116]:
X_valid_label0 = X_valid[X_valid_labels==0]
X_valid_label1 = X_valid[X_valid_labels==1]
X_valid_label2 = X_valid[X_valid_labels==2]
X_valid_label3 = X_valid[X_valid_labels==3]
X_valid_label4 = X_valid[X_valid_labels==4]

In [122]:
y_valid_label0 = y_valid[X_valid_labels==0]
y_valid_label1 = y_valid[X_valid_labels==1]
y_valid_label2 = y_valid[X_valid_labels==2]
y_valid_label3 = y_valid[X_valid_labels==3]
y_valid_label4 = y_valid[X_valid_labels==4]

In [123]:
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):
    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
        'booster' : 'gbtree',
        'max_depth' : int(max_depth),
        'gamma' : gamma,
        'eta' : 0.01,
        'objective' : 'binary:logistic',
        'nthread' : 4,
        'silent' : True,
        'eval_metric': 'logloss',
        'subsample' : max(min(subsample, 1), 0),
        'colsample_bytree' : max(min(colsample_bytree, 1), 0),
        'min_child_weight' : min_child_weight,
        'max_delta_step' : int(max_delta_step),
        'seed' : 1001
        }

    folds = 5
    cv_score = 0

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 500,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 20,
                    metrics = 'logloss',
                    show_stdv = True
               )

    val_score = xgbc['test-logloss-mean'].iloc[-1]
    train_score = xgbc['train-logloss-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
    (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)
    return (val_score*2) - 1

In [124]:
from bayes_opt import BayesianOptimization
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 50.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [125]:
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')

In [127]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    AUCbest = -1.
    ITERbest = 0
    y_dtrain=pd.get_dummies(y_train_label0)
    dtrain = xgb.DMatrix(X_train_label0, label = y_dtrain['Y'])
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.462092 val-auc = 0.475402 ( diff = -0.013310 ) train-gini = -0.075816 val-gini = -0.049197
    1 | 04m20s |   -0.04920 |             0.6899 |   14.2744 |           5.3743 |      9.1255 |             2.1522 |      0.7918 | 
 Stopped after 500 iterations with train-auc = 0.555306 val-auc = 0.556916 ( diff = -0.001610 ) train-gini = 0.110611 val-gini = 0.113832
    2 | 01m08s |    0.11383 |             0.4775 |   39.6686 |           7.4267 |      3.0898 |            19.4955 |      0.9017 | 
Bayesian Optimization
---------------------------

In [129]:
model = []

In [130]:
model.append(XGB_BO.res['max']['max_params'])

In [131]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    AUCbest = -1.
    ITERbest = 0
    y_dtrain=pd.get_dummies(y_train_label1)
    dtrain = xgb.DMatrix(X_train_label1, label = y_dtrain['Y'])
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.554025 val-auc = 0.554755 ( diff = -0.000731 ) train-gini = 0.108050 val-gini = 0.109511
    8 | 01m43s |    0.10951 |             0.4221 |    0.1476 |           1.1183 |      2.0471 |            19.2493 |      0.7121 | 
 Stopped after 500 iterations with train-auc = 0.554653 val-auc = 0.555362 ( diff = -0.000710 ) train-gini = 0.109306 val-gini = 0.110725
    9 | 01m36s |    0.11072 |             0.4474 |   48.2973 |           0.2069 |      2.8557 |            19.8383 |      0.8732 | 
 Stopped after 500 iterations with train-auc

In [132]:
model.append(XGB_BO.res['max']['max_params'])

In [133]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    AUCbest = -1.
    ITERbest = 0
    y_dtrain=pd.get_dummies(y_train_label2)
    dtrain = xgb.DMatrix(X_train_label2, label = y_dtrain['Y'])
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.538398 val-auc = 0.540633 ( diff = -0.002235 ) train-gini = 0.076796 val-gini = 0.081266
   13 | 01m21s |    0.08127 |             0.4000 |    0.0010 |          10.0000 |      2.0000 |             0.0000 |      0.4000 | 
 Stopped after 500 iterations with train-auc = 0.552250 val-auc = 0.553369 ( diff = -0.001120 ) train-gini = 0.104499 val-gini = 0.106738
   14 | 01m21s |    0.10674 |             0.4000 |   31.6167 |          10.0000 |      2.0000 |             8.9546 |      0.4000 | 
 Stopped after 500 iterations with train-auc

In [134]:
model.append(XGB_BO.res['max']['max_params'])

In [135]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    AUCbest = -1.
    ITERbest = 0
    y_dtrain=pd.get_dummies(y_train_label3)
    dtrain = xgb.DMatrix(X_train_label3, label = y_dtrain['Y'])
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.590825 val-auc = 0.591426 ( diff = -0.000601 ) train-gini = 0.181649 val-gini = 0.182852
   18 | 01m07s |    0.18285 |             0.4000 |   50.0000 |           0.0000 |      2.0000 |             0.0000 |      0.4000 | 
 Stopped after 500 iterations with train-auc = 0.584010 val-auc = 0.584987 ( diff = -0.000977 ) train-gini = 0.168020 val-gini = 0.169973
   19 | 01m03s |    0.16997 |             0.4000 |   32.6613 |          10.0000 |      2.0000 |            20.0000 |      0.4000 | 
 Stopped after 500 iterations with train-auc

In [136]:
model.append(XGB_BO.res['max']['max_params'])

In [138]:
t1=time.time()
import xgboost as xgb
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    AUCbest = -1.
    ITERbest = 0
    y_dtrain=pd.get_dummies(y_train_label4)
    dtrain = xgb.DMatrix(X_train_label4, label = y_dtrain['N'])
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Bayesian Optimization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 1 iterations with train-auc = 0.693147 val-auc = 0.693147 ( diff = 0.000000 ) train-gini = 0.386294 val-gini = 0.386294
   23 | 00m19s |    0.38629 |             0.4000 |   38.2139 |          10.0000 |      2.0000 |            20.0000 |      0.4000 | 
 Stopped after 500 iterations with train-auc = 0.157317 val-auc = 0.157317 ( diff = 0.000000 ) train-gini = -0.685366 val-gini = -0.685366
   24 | 00m30s |   -0.68537 |             1.0000 |   34.9275 |          10.0000 |     12.0000 |             0.0000 |      0.4000 | 
 Stopped after 1 iterations with train-auc = 0

In [139]:
model.append(XGB_BO.res['max']['max_params'])

In [140]:
model

[{'colsample_bytree': 0.4057543205443876,
  'gamma': 49.66951703468305,
  'max_delta_step': 0.2536929836930568,
  'max_depth': 2.6480271468430843,
  'min_child_weight': 19.688137921418548,
  'subsample': 0.42180164965893374},
 {'colsample_bytree': 0.4057543205443876,
  'gamma': 49.66951703468305,
  'max_delta_step': 0.2536929836930568,
  'max_depth': 2.6480271468430843,
  'min_child_weight': 19.688137921418548,
  'subsample': 0.42180164965893374},
 {'colsample_bytree': 0.4057543205443876,
  'gamma': 49.66951703468305,
  'max_delta_step': 0.2536929836930568,
  'max_depth': 2.6480271468430843,
  'min_child_weight': 19.688137921418548,
  'subsample': 0.42180164965893374},
 {'colsample_bytree': 0.4,
  'gamma': 50.0,
  'max_delta_step': 0.0,
  'max_depth': 2.0,
  'min_child_weight': 0.0,
  'subsample': 0.4},
 {'colsample_bytree': 0.4,
  'gamma': 38.21386237253613,
  'max_delta_step': 10.0,
  'max_depth': 2.0,
  'min_child_weight': 20.0,
  'subsample': 0.4}]

In [151]:
t1=time.time()
m_params=model[0]
xgb0=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred0=xgb0.fit(X_train_label0,y_train_label0).predict_proba(X_valid_label0)
print(log_loss(y_valid_label0,pred0[:,1]))
t2=time.time()
t2-t1

0.5896937829689518


10.020084381103516

In [143]:
t1=time.time()
m_params=model[1]
xgb1=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred1=xgb1.fit(X_train_label1,y_train_label1).predict_proba(X_valid_label1)
print(log_loss(y_valid_label1,pred1[:,1]))
t2=time.time()
t2-t1

0.5620942755784056


18.357664585113525

In [144]:
t1=time.time()
m_params=model[2]
xgb2=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred2=xgb2.fit(X_train_label2,y_train_label2).predict_proba(X_valid_label2)
print(log_loss(y_valid_label2,pred2[:,1]))
t2=time.time()
t2-t1

0.5663942319960743


5.791574001312256

In [145]:
t1=time.time()
m_params=model[3]
xgb3=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred3=xgb3.fit(X_train_label3,y_train_label3).predict_proba(X_valid_label3)
print(log_loss(y_valid_label3,pred3[:,1]))
t2=time.time()
t2-t1

0.5866473625850822


4.001896381378174

In [149]:
t1=time.time()
m_params=model[4]
xgb4=xgb.XGBClassifier(max_depth=int(m_params['max_depth']),gamma=m_params['gamma'],colsample_bytree=m_params['colsample_bytree'],
                       eval_metric='logloss',max_delta_step= m_params['max_delta_step'],min_child_weight=m_params['min_child_weight'],
                      subsample= m_params['subsample'])
pred4=xgb4.fit(X_train_label4,y_train_label4).predict_proba(X_valid_label4)
print(log_loss(y_valid_label4,pred4[:]))
t2=time.time()
t2-t1

ValueError: y_true contains only one label (N). Please provide the true labels explicitly through the labels argument.

In [150]:
print(pred4)

[[0.5 0.5]
 [0.5 0.5]]


In [169]:
pred = np.zeros(len(y_valid))
pred[X_valid_labels==0] = pred0[:,1]
pred[X_valid_labels==1] = pred1[:,1]
pred[X_valid_labels==2] = pred2[:,1]
pred[X_valid_labels==3] = pred3[:,1]
pred[X_valid_labels==4] = 0.5

In [170]:
print(log_loss(y_valid,pred))

0.5723900402884352


In [172]:
test_labels=kmeans.predict(test)

In [175]:
pred = np.zeros(test.shape[0])
test_label0=test[test_labels==0]
test_label1=test[test_labels==1]
test_label2=test[test_labels==2]
test_label3=test[test_labels==3]
test_label4=test[test_labels==4]

In [176]:
pred[test_labels==0] = xgb0.predict_proba(test_label0)[:,1]
pred[test_labels==1] = xgb1.predict_proba(test_label1)[:,1]
pred[test_labels==2] = xgb2.predict_proba(test_label2)[:,1]
pred[test_labels==3] = xgb3.predict_proba(test_label3)[:,1]
pred[test_labels==4] = 0.5

In [179]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-BO-multimodel_imputed.csv',index=False)

In [186]:
(pred>0.7).sum()

1151916